In [3]:
# Imports
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc, Input, Output
import numpy as np
import dash_bootstrap_components as dbc
import dash_daq as daq
from simulation.functions import runSimulation, aggregateOutputs, readData
import os
import sys

# Datasets
datasets = readData(area="kamppi")
dataset_names = list(datasets.keys())
max_sim_timestep = datasets["Baseline emissions"]["Simulation timestep"].max()/60
timeline = np.arange(1, max_sim_timestep+1, 1)
####-----------------------------------------

# Global variables
mobility_modes = ["Pedestrians", "Private cars", "Bicycles", "Buses", "Trams"]
objectives = ["Summary", "Air quality", "Livability", "Traffic"]
seasons = ["Winter", "Summer"]
weekdays = ["Weekday", "Weekend"]
mobility_demand = ["Low", "Realistic", "High"]
areas = ["Arabia", "Jätkäsaari", "Kamppi", "Vihdintie"]

####-----------------------------------------

# From variable-situation-to-dataset-columns mapping
# (Situation, variable) -> (dataset, [list of columns])
from_situ_var_to_data_cols = {
    ("Baseline", "Trips"): (
        "Baseline trips",
        ["Travel time", "Lost time", "Mobility mode", "Mobility flow",],
    ),
    ("Baseline", "Mobility flow"): (
        "Baseline emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Mobility flow",
            "Mobility mode",
        ],
    ),
    ("Baseline", "Speed"): (
        "Baseline emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Mobility flow",
            "Mobility mode",
            "Speed",
        ],
    ),
    ("Baseline", "Travel time"): (
        "Baseline trips",
        ["Mobility mode", "Travel time", "Mobility flow",],
    ),
    ("Baseline", "Lost time"): (
        "Baseline trips",
        ["Mobility mode", "Lost time", "Mobility flow",],
    ),
    ("Baseline", "Noise"): (
        "Baseline noise",
        ["Name", "Edge", "Simulation timestep", "Longitude", "Latitude", "Noise", "Mobility flow"],
    ),
    ("Optimized", "Mobility flow"): (
        "Optimized emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Mobility flow",
            "Mobility mode",
        ],
    ),
    ("Optimized", "Speed"): (
        "Optimized emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Mobility flow",
            "Mobility mode",
            "Speed",
        ],
    ),
    ("Optimized", "Travel time"): (
        "Optimized trips",
        ["Mobility mode", "Travel time", "Mobility flow",],
    ),
    ("Optimized", "Noise"): (
        "Optimized noise",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Noise",
            "Mobility flow",
        ],
    ),
    ("Baseline", "Carbon monoxide"): (
        "Baseline emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Carbon monoxide",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Baseline", "Carbon dioxide"): (
        "Baseline emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Carbon dioxide",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Baseline", "Hydrocarbon"): (
        "Baseline emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Hydrocarbon",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Baseline", "Nitrogen oxides"): (
        "Baseline emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Nitrogen oxides",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Baseline", "Respirable particles"): (
        "Baseline emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Respirable particles",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Baseline", "Fine particles"): (
        "Baseline emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Fine particles",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Optimized", "Carbon monoxide"): (
        "Optimized emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Carbon monoxide",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Optimized", "Carbon dioxide"): (
        "Optimized emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Carbon dioxide",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Optimized", "Hydrocarbon"): (
        "Optimized emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Hydrocarbon",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Optimized", "Nitrogen oxides"): (
        "Optimized emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Nitrogen oxides",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Optimized", "Respirable particles"): (
        "Optimized emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Respirable particles",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Optimized", "Fine particles"): (
        "Optimized emissions",
        [
            "Name",
            "Edge",
            "Simulation timestep",
            "Longitude",
            "Latitude",
            "Fine particles",
            "Mobility mode",
            "Mobility flow",
        ],
    ),
    ("Baseline", "Relocation rate"): (
        "Baseline livability",
        ["Name", "Longitude", "Latitude", "Relocation rate",],
    ),
    ("Optimized", "Relocation rate"): (
        "Optimized livability",
        ["Name", "Longitude", "Latitude", "Relocation rate",],
    ),
}

timeline_functions = {"Average": "mean", "Sum": "sum"}

####-----------------------------------------

# Traffic variables
traffic_variables = ["Mobility flow",
                     "Speed",
                     "Travel time",
                     "Lost time",
                     "Noise"]
traffic_units = {"Mobility flow": "# of passengers",
                "Speed": "m/s",
                "Travel time": "# of trips",
                "Lost time": "# of trips",
                "Noise": "dB"}

####-----------------------------------------

# AQ variables
aqi_pm25_thresholds = [0, 10, 25, 50, 75, np.inf]
aqi_labels = ["good", "satisfactory", "fair", "poor", "very poor"]
aq_variables = ["Carbon monoxide",
                "Carbon dioxide",
                "Hydrocarbon",
                "Nitrogen oxides",
                "Respirable particles",
                "Fine particles"]
aq_units = {
"Carbon monoxide": "mg",
"Carbon dioxide": "mg",
"Hydrocarbon": "mg",
"Nitrogen oxides": "mg",
"Respirable particles": "µg",
"Fine particles": "µg"
}

####-----------------------------------------

# Livability variables

####-----------------------------------------

# Re-index timesteps from simulation seconds to minutes
def new_timeline(network, timestep_range):
    timesteps = np.arange(network["Simulation timestep"].min(), network["Simulation timestep"].max(), timestep_range*60, dtype=int)
    timestep_labels = timesteps//60
    network["Timestep"] = pd.cut(network["Simulation timestep"], 
                                bins=timesteps,
                                labels=timestep_labels[1:],
                                right=False, 
                                include_lowest=True)
    return network

def map_bounds(network):
    west_bound = network["Longitude"].min()
    east_bound = network["Longitude"].max()
    south_bound = network["Latitude"].min()
    north_bound = network["Latitude"].max()
    map_bounds = {
        "west": west_bound * (1 - 2e-4),
        "east": east_bound * (1 + 2e-4),
        "south": south_bound * (1 - 5e-6),
        "north": north_bound * (1 + 5e-6),
    }
    return map_bounds

# Set the location text and spatial scope for the data when the location is changed
def spatial_scope(spatial_click_data, network):
    if spatial_click_data is None:
        location = "Location: Network"
    else:
        road_lon = spatial_click_data["points"][0]["lon"]
        road_lat = spatial_click_data["points"][0]["lat"]
        name = spatial_click_data["points"][0]["customdata"][0]
        location = f"Location: {name} ({road_lat:.2f} °N, {road_lon:.2f} °E)"
        edge = spatial_click_data["points"][0]["customdata"][1]
        network = network[network["Edge"] == edge]
    return network, location

def copy_data(situation, variable):
    dataset_name, required_columns = from_situ_var_to_data_cols[(situation, variable)]
    data = datasets[dataset_name]
    # Make a copy of the required data (to avoid changing the simulation output)
    network = data[required_columns].copy()
    return network

# Helper function to normalize to an array to a range (x,y)
def normalize_range(array, x, y):
    m = min(array)
    range = max(array) - m
    array = (array - m)/range
    range2 = y - x
    normalized = (array*range2) + x
    return list(normalized)

####-----------------------------------------

# App

# Style
external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.icons.BOOTSTRAP]

# Hover layout
hovers=dict(bgcolor="white", font_size=16)

# Main app
app = Dash(__name__,
           external_stylesheets=external_stylesheets,
           suppress_callback_exceptions=True)

# Main layout
app.layout = html.Div(
    [
        # Page title
        html.H1("AI-based optimisation tool for sustainable urban planning (AIOSut)", style={"padding-top": "2vh", "padding-bottom": "4vh"}),

        # Dividing line between the header and interfaces
        html.Hr(),

        # Scenario params
        html.Div(
            [
                # Title
                html.H2("Scenario parameters", style={"display": "inline-block"}),
                dbc.Row(
                    [
                        dbc.Col(
                            # Buttons for season
                            html.Div(
                                [
                                    html.Label(
                                        "Season",
                                        style={
                                            "font-size": 24,
                                            "padding-bottom": "2vh",
                                            "padding-top": "1vh",
                                        },
                                    ),
                                    dcc.Dropdown(
                                        options=seasons,
                                        value="Summer",
                                        id="crossfilter-season",
                                        style={
                                            "font-size": 18,
                                            "width": "95%",
                                        },
                                    ),
                                ],
                                style={"padding-top": "2vh", "padding-bottom": "2vh"},
                            ),
                        ),
                        dbc.Col(
                            # Dropdown for weekday
                            html.Div(
                                [
                                    html.Label(
                                        "Weekday",
                                        style={
                                            "font-size": 24,
                                            "padding-bottom": "2vh",
                                            "padding-top": "1vh",
                                        },
                                    ),
                                    dcc.Dropdown(
                                        options=weekdays,
                                        value="Weekday",
                                        id="crossfilter-day",
                                        style={
                                            "font-size": 18,
                                            "width": "95%",
                                        },
                                    ),
                                ],
                                style={"padding-top": "2vh", "padding-bottom": "2vh"},
                            ),
                        ),
                        dbc.Col(
                            # Buttons for traffic demand
                            html.Div(
                                [
                                    html.Label(
                                        "Demand",
                                        style={
                                            "font-size": 24,
                                            "padding-bottom": "2vh",
                                            "padding-top": "1vh",
                                        },
                                    ),
                                    dcc.Dropdown(
                                        options=mobility_demand,
                                        value="Realistic",
                                        id="crossfilter-mobility-demand",
                                        style={
                                            "font-size": 18,
                                            "width": "95%",
                                        },
                                    ),
                                ],
                                style={"padding-top": "2vh", "padding-bottom": "2vh"},
                            ),
                        ),
                        dbc.Col(
                            # Dropdown for area
                            html.Div(
                                [
                                    html.Label(
                                        "Area",
                                        style={
                                            "font-size": 24,
                                            "padding-bottom": "2vh",
                                            "padding-top": "1vh",
                                        },
                                    ),
                                    dcc.Dropdown(
                                        options=areas,
                                        value="Kamppi",
                                        id="crossfilter-area",
                                        style={
                                            "font-size": 18,
                                            "width": "95%",
                                        },
                                    ),
                                ],
                                style={"padding-top": "2vh", "padding-bottom": "2vh"},
                            ),
                        ),
                    ]
                ),
            ],
        ),
        # Dividing line between the parameters
        html.Hr(),
        # Title
        html.H2(
            "Optimization parameters",
            style={"padding-top": "2vh", "padding-bottom": "2vh"},
        ),
        # Optimization params
        html.Div(
            [
                dbc.Row(
                    [
                        dbc.Col(
                            html.Label(
                                "Objective",
                            ),
                            width={"size": 2},
                        ),
                        dbc.Col(
                            html.Center(
                                html.Label(
                                    "Priority",
                                ),
                            ),
                        ),
                        dbc.Col(
                            html.Center(
                                html.Label(
                                    "Performance thresholds",
                                ),
                            ),
                        ),
                    ],
                    style={
                        "padding-bottom": "2vh",
                        "padding-top": "1vh",
                        "font-size": 24,
                    },
                ),
                dbc.Row(
                    [
                        dbc.Col(
                            html.Label(
                                "Traffic",
                                style={
                                    "font-size": 18,
                                },
                            ),
                            width={"size": 2},
                        ),
                        dbc.Col(
                            dcc.Slider(
                                min=0,
                                max=100,
                                step=10,
                                value=50,
                                id="traffic-optim-slider",
                            ), align="end",
                        ),
                        dbc.Col(
                            html.Center(
                                html.Div(
                                    [
                                        daq.NumericInput(
                                            value=100,
                                            id="crossfilter-traffic-optim-threshold",
                                            min=0,
                                            max=100,
                                            label="% of improvement",
                                            labelPosition="bottom",
                                            style={
                                                "display": "inline-block",
                                                "font-size": 18,
                                                "padding-right": "1vw",
                                            },
                                        ),
                                    ]
                                ),
                            ),
                        ),
                    ],
                    style={
                        "font-size": 18,
                        "padding-bottom": "2vh",
                        "padding-top": "2vh",
                    },
                    align="center",
                ),
                dbc.Row(
                    [
                        dbc.Col(
                            html.Label(
                                "Air quality",
                                style={
                                    "font-size": 18,
                                },
                            ),
                            width={"size": 2},
                        ),
                        dbc.Col(
                            dcc.Slider(
                                min=0,
                                max=100,
                                step=10,
                                value=50,
                                id="air-quality-optim-slider",
                            ), align="end",
                        ),
                        dbc.Col(
                            html.Center(
                                html.Div(
                                    [
                                        daq.NumericInput(
                                            value=100,
                                            id="crossfilter-aq-optim-threshold",
                                            min=0,
                                            max=100,
                                            label="% of improvement",
                                            labelPosition="bottom",
                                            style={
                                                "display": "inline-block",
                                                "font-size": 18,
                                                "padding-right": "1vw",
                                            },
                                        ),
                                    ]
                                ),
                            ),
                        ),
                    ],
                    style={
                        "font-size": 18,
                        "padding-bottom": "2vh",
                        "padding-top": "2vh",
                    },
                    align="center",
                ),
                dbc.Row(
                    [
                        dbc.Col(
                            html.Label(
                                "Livability",
                                style={
                                    "font-size": 18,
                                },
                            ),
                            width={"size": 2},
                        ),
                        dbc.Col(
                            dcc.Slider(
                                min=0,
                                max=100,
                                step=10,
                                value=50,
                                id="livability-optim-slider",
                            ), align="end",
                        ),
                        dbc.Col(
                            html.Center(
                                html.Div(
                                    [
                                        daq.NumericInput(
                                            value=100,
                                            id="crossfilter-liv-optim-threshold",
                                            label="% of improvement",
                                            labelPosition="bottom",
                                            min=0,
                                            max=100,
                                            style={
                                                "display": "inline-block",
                                                "font-size": 18,
                                                "padding-right": "1vw",
                                            },
                                        ),
                                    ]
                                ),
                            ),
                        ),
                    ],
                    style={
                        "font-size": 18,
                        "padding-bottom": "1vh",
                        "padding-top": "1vh",
                    },
                    align="center",
                ),
            ]
        ),
        # Simulate button
        html.Div(
            html.Center(
                dbc.Button(
                    html.Label(
                        "Simulate",
                        style={
                            "font-size": 18,
                        },
                    ),
                    size="lg",
                ),
                style={
                    "padding-top": "2vh",
                    "padding-bottom": "2vh",
                    "display": "block",
                },
            ), style={"padding-top": "3vh"}
        ),
        # Dividing line between the parameters
        html.Hr(),
        # Viz params
        html.Div(
            [
                # Title
                html.H2(
                    "Visualization parameters",
                    style={"padding-top": "2vh", "padding-bottom": "2vh"},
                ),
            ]
        ),
        # Toggles and texts for parameters
        dbc.Row(
            [
                # Timeline type column
                dbc.Col(
                    html.Div(
                        [
                            html.Label(
                                "Temporal aggregation", style={"padding-bottom": "2vh"}
                            ),
                            dcc.RadioItems(
                                options=["Sum", "Average"],
                                value="Sum",
                                id="crossfilter-timeline-type",
                                inline=True,
                                labelStyle={
                                    "automargin": True,
                                    "padding-top": "1vh",
                                    "font-size": 18,
                                    "padding-right": "1vw",
                                },
                                inputStyle={"margin-right": "0.3vw"},
                            ),
                        ]
                    )
                ),
                # Timestep interval column
                dbc.Col(
                    html.Div(
                        [
                            html.Label(
                                "Timestep interval", style={"padding-bottom": "2vh"}
                            ),
                            html.Div(
                                [
                                    daq.NumericInput(
                                        value=1,
                                        id="crossfilter-timestep-range",
                                        label="minute(s)",
                                        labelPosition="bottom",
                                        min=1,
                                        max=timeline.max(),
                                        style={
                                            "display": "inline-block",
                                            "font-size": 18,
                                            "padding-right": "1vw",
                                        },
                                    ),
                                ]
                            ),
                        ]
                    )
                ),
                # Row end
            ],
            justify="center",
            style={"automargin": True, "padding-top": "2vh", "font-size": 24},
        ),
        # Dividing line between the params and plots
        html.Hr(),
        # Title
        html.H2("Results", style={"padding-bottom": "2vh", "padding-top": "1vh"}),
        # Tabs
        dcc.Tabs(
            id="tabs-1",
            value="traffic",
            children=[
                dcc.Tab(label="Summary", value="summary"),
                dcc.Tab(label="Traffic", value="traffic"),
                dcc.Tab(label="Air quality", value="air-quality"),
                dcc.Tab(label="Livability", value="livability"),
            ],
        ),
        html.Div(id="tabs-content-1", style={"padding-top": "1vh"}),
        # Main layout style
    ],
    style={"width": "99vw", "automargin": True, "padding": "5vh 5vh 5vh 5vh"},
)

####-----------------------------------------

# Callbacks

# Tab update callback
@app.callback(
    Output('tabs-content-1', 'children'),
    Input('tabs-1', 'value')
)
def renderViewContent(tab):

    ####-----------------------------------------
    # View for summary tab
    if tab == 'summary':
        return html.Label("Summary under work", style={"padding-top": "2vh", "font-size": 20})

    ####-----------------------------------------
    # View for traffic tab
    elif tab == 'traffic':
        return html.Center(
            [
            # Current or optimized situation
            html.Div(
                [
                    html.Label(
                        "Situation",
                        style={
                            "font-size": 24,
                            "padding-bottom": "2vh",
                            "padding-top": "1vh",
                        },
                    ),
                    dcc.RadioItems(
                        options=["Baseline", "Optimized"],
                        value="Baseline",
                        id="crossfilter-situation-traffic",
                        inline=True,
                        labelStyle={
                            "automargin": True,
                            "padding-top": "1vh",
                            "font-size": 18,
                            "padding-right": "1vw",
                        },
                        inputStyle={"margin-right": "0.4vw"},
                    ),
                ],
                style={"padding-top": "2vh", "padding-bottom": "2vh"},
            ),
              
            # Variable dropdown
            html.Div(
                [
                html.Label(
                    "Variable",
                    style={
                        "font-size": 24,
                        "padding-bottom": "2vh",
                        "padding-top": "1vh",
                    },
                ),
                    dcc.Dropdown(
                        options=traffic_variables,
                        value="Mobility flow",
                        id="crossfilter-variable-traffic",
                        style={
                            "font-size": 18,
                            "width": "50%",
                            "margin": "auto",
                            "text-align": "center",
                        },
                    ),
                ],
                style={"padding-top": "2vh", "padding-bottom": "2vh"},
            ),

            # The heatmap
            html.Div(
                [
                    dcc.Loading(
                        dcc.Graph(
                            id="traffic-one",
                            responsive=True,
                            style={
                                "height": "45vw",
                                "width": "68vw",
                            },
                            config={"showTips": True},
                        ),
                        type="cube",
                    ),
                    # Title
                    html.Label(
                        html.Pre(id="location-text-traffic"),
                        style={
                            "font-size": 24,
                            "padding-bottom": "4vh",
                            "display": "block",
                            "padding-top": "2vh",
                        },
                    ),
                    # Reset button
                    dbc.Button(
                        "Reset location",
                        id="reset-button-traffic",
                        n_clicks=0,
                        style={
                            "display": "block",
                            "font-size": 18,
                            "automargin": True,
                            "padding": "1vh",
                        },
                    ),
                ],
                id = "traffic-one-div",
            ),

            # Bar plot
            html.Div(
                dcc.Loading(
                    dcc.Graph(
                        id="traffic-two",
                        responsive=True,
                        style={"height": "30vw"},
                    ),
                    type="cube",
                ),
                id = "traffic-two-div",
            ),

            # Stream graph
            html.Div(
                    dcc.Loading(
                        dcc.Graph(
                            id="traffic-three",
                            responsive=True,
                            style={"height": "30vw"},
                        ),
                        type="cube",
                    ),
                id = "traffic-three-div",
            ),
        ]
    ),

    ####-----------------------------------------
    # View for AQ tab
    elif tab == 'air-quality':
        return html.Center(
            [
            # Current or optimized situation
            html.Div(
                [
                    html.Label(
                        "Situation",
                        style={
                            "font-size": 24,
                            "padding-bottom": "2vh",
                            "padding-top": "1vh",
                        },
                    ),
                    dcc.RadioItems(
                        options=["Baseline", "Optimized"],
                        value="Baseline",
                        id="crossfilter-situation-aq",
                        inline=True,
                        labelStyle={
                            "automargin": True,
                            "padding-top": "1vh",
                            "font-size": 18,
                            "padding-right": "1vw",
                        },
                        inputStyle={"margin-right": "0.4vw"},
                    ),
                ],
                style={"padding-top": "2vh", "padding-bottom": "2vh"},
            ),
            # Variable dropdown
            html.Div(
                [
                    html.Label(
                        "Variable",
                        style={
                            "font-size": 24,
                            "padding-bottom": "2vh",
                            "padding-top": "1vh",
                        },
                    ),
                    dcc.Dropdown(
                        options=aq_variables,
                        value="Carbon monoxide",
                        id="crossfilter-variable-aq",
                        style={
                            "font-size": 18,
                            "width": "50%",
                            "margin": "auto",
                            "text-align": "center",
                        },
                    ),
                ],
                style={"padding-top": "2vh", "padding-bottom": "2vh"},
            ),
            # Network plot
            html.Div(
                [
                    # The heatmap
                    dcc.Loading(
                        dcc.Graph(
                            id="aq-one",
                            responsive=True,
                            config={"showTips": True},
                            style={"height": "45vw", "width": "68vw"},
                        ),
                        type="cube",
                    ),
                    # Title
                    html.Label(
                        html.Pre(id="location-text-aq"),
                        style={
                            "font-size": 24,
                            "padding-bottom": "4vh",
                            "display": "block",
                            "padding-top": "2vh",
                        },
                    ),
                    # Reset button
                    dbc.Button(
                        "Reset location",
                        id="reset-button-aq",
                        n_clicks=0,
                        style={
                            "display": "block",
                            "font-size": 18,
                            "automargin": True,
                            "padding": "1vh",
                        },
                    ),
                ],
                id = "aq-one-div",
            ),
            # Bar plot
            html.Div(     
                dcc.Loading(
                    dcc.Graph(
                        id="aq-two",
                        responsive=True,
                        config={"showTips": True},
                        style={"height": "30vw"},
                    ),
                    type="cube",
                ),
                id = "aq-two-div",
            ),
            # Stream graph
            html.Div(
                dcc.Loading(
                    dcc.Graph(
                        id="aq-three",
                        responsive=True,
                        style={"height": "30vw"},
                    ),
                    type="cube",
                ),
                id = "aq-three-div"
            ),
        ],
    ),

    ####-----------------------------------------
    # View for livability tab
    elif tab == 'livability':

        return html.Center(
            [
            # Current or optimized situation
            html.Div([
                html.Label("Situation", style={"font-size": 24, "padding-bottom": "2vh", "padding-top": "1vh"}),
                dcc.RadioItems(options=["Baseline", "Optimized"],
                    value="Baseline",
                    id="crossfilter-situation",
                    inline=True, 
                    labelStyle={"automargin": True, "padding-top": "1vh", "font-size": 18, "padding-right": "1vw"}, 
                    inputStyle={"margin-right": "0.4vw"}),                
                ],
                style={"padding-top": "2vh", "padding-bottom": "2vh"}
            ),
                
            html.Label("Livability under work", style={"padding-top": "2vh", "font-size": 20}),

            ]
        )


####-----------------------------------------

## Traffic


# Callback to traffic heatmap
@app.callback(
    Output("traffic-one", "figure"),
    Output("traffic-one-div", "style"),
    Output("traffic-two", "figure"),
    Output("traffic-two-div", "style"),
    Output("traffic-three", "figure"),
    Output("traffic-three-div", "style"),
    Output("location-text-traffic", "children"),
    Input("crossfilter-situation-traffic", "value"),
    Input("crossfilter-variable-traffic", "value"),
    Input("crossfilter-timestep-range", "value"),
    Input("crossfilter-timeline-type", "value"),
    Input("traffic-one", "clickData"),
)
def drawTraffic(situation, variable, timestep_range, timeline_type, spatial_click_data):
    first_plot = {}
    second_plot = {}
    third_plot = {}
    first_plot_style = {"display": "none"}
    second_plot_style = {"display": "none"}
    third_plot_style = {"display": "none"}
    location = ""
    # Heatmaps
    if variable in ["Mobility flow"]:
        # Make a copy of the required data (to avoid changing the simulation output)
        network = copy_data(situation=situation, variable=variable)
        network = new_timeline(network=network, timestep_range=timestep_range)
        network, location = spatial_scope(spatial_click_data, network)
        bounds = map_bounds(network)

        # Draw the heatmap
        heatmap_network = (
                network.groupby(["Timestep", "Edge"], observed=False)
                .agg(
                    {
                        variable: timeline_functions[timeline_type],
                        "Longitude": "first",
                        "Latitude": "first",
                        "Name": "first",
                    }
                )
                .reset_index()
            )
        # Draw the figure
        legend_max = 1.2 * heatmap_network[heatmap_network[variable] != 0][variable].median()
        heatmap = px.density_map(
            data_frame=heatmap_network,
            lat="Latitude",
            lon="Longitude",
            z=variable,
            radius=15,
            map_style="open-street-map",
            range_color=(0, legend_max),
            hover_data={
                "Name": True,
                "Edge": False,
                "Mobility flow": True,
                "Longitude": False,
                "Latitude": False,
                variable: True,
            },
            # labels={traffic_column: variable},
            animation_frame="Timestep",
            title="Network heatmap",
        )
        # Customize the sliders with a larger font size and prefix, retain the active frame
        sliders = [dict(
        currentvalue={"prefix": "Time (in minutes): "},
        font={"size": 14},
        pad={"t": 50}
        )]
        # Update points' opacity on click
        heatmap.update_layout(
            clickmode="event+select",
            sliders=sliders,
            map_bounds=bounds
        )
        # Sets the variable name and its unit as a legend to the color bar
        heatmap.layout["coloraxis"]["colorbar"]["title"] = f"{variable} ({traffic_units[variable]})"
        first_plot = heatmap
        first_plot_style = {"padding-top": "3vh"}

        ## Draw the area chart
        area_network = network.groupby(["Mobility mode", "Timestep"], observed=False).agg(
                {variable: timeline_functions[timeline_type]}
                ).reset_index()
        # Draw the figure
        area_plot = px.area(
                data_frame=area_network,
                x="Timestep",
                y=variable,
                color="Mobility mode",
                color_discrete_sequence=px.colors.sequential.Plasma_r,
                title="Mobility mode time series",
            )
        # Customize the hovers, title and yaxis
        area_plot.update_layout(hoverlabel=hovers,
        title_x=0.11,
        yaxis_title=f"{variable} ({traffic_units[variable]})")
        third_plot = area_plot
        third_plot_style = {"padding-top": "3vh"}

    ## Bar plots
    elif variable in ["Travel time", "Lost time"]:
        barplot_network = copy_data(situation=situation, variable=variable)
        bar_plot = px.histogram(data_frame=barplot_network, x=variable, color="Mobility mode", barmode="overlay")
        bar_plot.update_layout(yaxis_title = traffic_units[variable], bargap=0.2, xaxis_title = f"{variable} in seconds")
        second_plot = bar_plot
        second_plot_style = {"padding-top": "3vh"}

    elif variable in ["Noise", "Speed"]:
        ## Make a copy of the initial data (to avoid changing the simulation output)
        network = copy_data(situation=situation, variable=variable)
        network = new_timeline(network=network, timestep_range=timestep_range)
        network, location = spatial_scope(spatial_click_data, network)
        bounds = map_bounds(network)

        ## Draw the heatmap
        heatmap_network = (
                    network.groupby(["Timestep", "Edge"], observed=False)
                    .agg(
                        {
                            variable: timeline_functions[timeline_type],
                            "Longitude": "first",
                            "Latitude": "first",
                            "Name": "first",
                            "Mobility flow": timeline_functions[timeline_type],
                        }
                    )
                    .reset_index()
                )
        legend_max = 1.2 * heatmap_network[heatmap_network[variable] != 0][variable].median()
        heatmap = px.density_map(data_frame=heatmap_network,
        lat = "Latitude",
        lon = "Longitude",
        z = variable,
        radius=15,
        zoom=10,
        range_color = (0,legend_max),
        map_style="open-street-map",
        hover_data = {"Name": True, 
                    "Edge": False,
                    "Mobility flow": True, 
                    "Longitude": False, 
                    "Latitude": False, 
                    variable: True},
        animation_frame="Timestep",
        title="Network heatmap"
        )
        # Customize the sliders with a larger font size and prefix
        sliders = [dict(
        currentvalue={"prefix": "Time (in minutes): "},
        # active=active_frame,
        font={"size": 14},
        pad={"t": 50}
        )]
        # Update points' opacity on click
        heatmap.update_layout(
            clickmode="event+select",
            sliders=sliders,
            map_bounds=bounds
        )
        # Sets the variable name and its unit as a legend to the color bar
        heatmap.layout["coloraxis"]["colorbar"]["title"] = f"{variable} ({traffic_units[variable]})"
        first_plot = heatmap
        first_plot_style = {"padding-top": "3vh"}

        ## Draw the bar plot
        secondary_network = datasets["Optimized emissions"].copy()
        barplot_network = (
            secondary_network.groupby(["Mobility mode"], observed=False)
            .agg({variable: "sum", "Mobility flow": "sum"})
            .reset_index()
        )
        barplot_network["Vehicle average"] = np.round(barplot_network[variable] / barplot_network["Mobility flow"], 4)
        bar_plot = px.bar(
            data_frame=barplot_network,
            x=variable,
            y="Mobility mode",
            color="Vehicle average",
            hover_data="Mobility flow",
            title="Vehicle averages per mobility mode",
        )
        # Customize the hovers, title, xaxis and legend
        bar_plot.update_layout(
            hoverlabel=hovers,
            title_x=0.11,
            bargap=0.5,
            xaxis_title=f"{variable} ({traffic_units[variable]})",
            yaxis_title="Mobility mode",
            xaxis=dict(range=[0, barplot_network[variable].max()*1.1]),
        )
        second_plot = bar_plot
        second_plot_style = {"padding-top": "3vh"}

        # Draw the area chart

        area_network = (
            network.groupby(["Timestep"], observed=False)
            .agg({variable: timeline_functions[timeline_type], "Mobility flow": timeline_functions[timeline_type]})
            .reset_index()
        )
        # Draw the figure
        area_plot = px.area(
            data_frame=area_network,
            x="Timestep",
            y=variable,
            color_discrete_sequence=px.colors.sequential.Plasma_r,
            hover_data=["Mobility flow"],
            title="Mobility mode time series",
        )
        # Customize the hovers, title and yaxis
        area_plot.update_layout(
            hoverlabel=hovers,
            title_x=0.11,
            yaxis_title=f"{variable} ({traffic_units[variable]})",
        )
        third_plot = area_plot
        third_plot_style = {"padding-top": "3vh"}

    # Return the figures, styles and location
    return first_plot, first_plot_style, second_plot, second_plot_style, third_plot, third_plot_style, location

# Reset the traffic heatmap
@app.callback(
    Output("traffic-one", "clickData"),
    [Input("reset-button-traffic", "n_clicks")])
def resetTrafficLocation(reset):
    return None

####-----------------------------------------

## Air quality


# Callback to AQ heatmap
@app.callback(
    Output("aq-one", "figure"),
    Output("aq-one", "style"),
    Output("aq-two", "figure"),
    Output("aq-two", "style"),
    Output("aq-three", "figure"),
    Output("aq-three", "style"),
    Output("location-text-aq", "children"),
    Input("crossfilter-situation-aq", "value"),
    Input("crossfilter-variable-aq", "value"),
    Input("crossfilter-timestep-range", "value"),
    Input("crossfilter-timeline-type", "value"),
    Input("aq-one", "clickData"),
)
def drawAirQuality(situation, variable, timestep_range, timeline_type, spatial_click_data):

    first_plot = {}
    second_plot = {}
    third_plot = {}
    first_plot_style = {"display": "none"}
    second_plot_style = {"display": "none"}
    third_plot_style = {"display": "none"}
    location = ""

    ## Make a copy of the initial data (to avoid changing the simulation output)
    network = copy_data(situation=situation, variable=variable)
    network = new_timeline(network=network, timestep_range=timestep_range)
    network, location = spatial_scope(spatial_click_data, network)
    bounds = map_bounds(network)

    ## Draw the heatmap
    heatmap_network = (
                network.groupby(["Timestep", "Edge"], observed=False)
                .agg(
                    {
                        variable: timeline_functions[timeline_type],
                        "Longitude": "first",
                        "Latitude": "first",
                        "Name": "first",
                        "Mobility flow": "sum",
                    }
                )
                .reset_index()
            )
    legend_max = 1.2 * heatmap_network[heatmap_network[variable] != 0][variable].median()
    heatmap = px.density_map(data_frame=heatmap_network,
    lat = "Latitude",
    lon = "Longitude",
    z = variable,
    radius=15,
    zoom=10,
    range_color = (0,legend_max),
    map_style="open-street-map",
    hover_data = {"Name": True, 
                  "Edge": False,
                  "Mobility flow": True,
                  "Longitude": False, 
                  "Latitude": False, 
                  variable: True},
    animation_frame="Timestep",
    title="Network heatmap"
    )
    # Customize the sliders with a larger font size and prefix
    sliders = [dict(
    currentvalue={"prefix": "Time (in minutes): "},
    # active=active_frame,
    font={"size": 14},
    pad={"t": 50}
    )]
    # Update points' opacity on click
    heatmap.update_layout(
        clickmode="event+select",
        sliders=sliders,
        map_bounds=bounds
    )
    # Sets the variable name and its unit as a legend to the color bar
    heatmap.layout["coloraxis"]["colorbar"]["title"] = f"{variable} ({aq_units[variable]})"
    first_plot = heatmap
    first_plot_style = {"padding-top": "3vh"}

    ## Draw the bar plot
    barplot_network = (
        network.groupby(["Mobility mode"], observed=False)
        .agg({variable: "sum", "Mobility flow": "sum"})
        .reset_index()
    )
    barplot_network["Vehicle average"] = np.round(barplot_network[variable] / barplot_network["Mobility flow"], 4)
    bar_plot = px.bar(
        data_frame=barplot_network,
        x=variable,
        y="Mobility mode",
        color="Vehicle average",
        hover_data="Mobility flow",
        title="Vehicle averages per mobility mode",
    )
    # Customize the hovers, title, xaxis and legend
    bar_plot.update_layout(
        hoverlabel=hovers,
        title_x=0.11,
        bargap=0.5,
        xaxis_title=f"{variable} ({aq_units[variable]})",
        yaxis_title="Mobility mode",
        xaxis=dict(range=[0, barplot_network[variable].max()]),
    )

    second_plot = bar_plot
    second_plot_style = {"padding-top": "3vh"}

    # Draw the area chart

    area_network = (
        network.groupby(["Mobility mode", "Timestep"], observed=False)
        .agg({variable: timeline_functions[timeline_type], "Mobility flow": timeline_functions[timeline_type]})
        .reset_index()
    )
    # Draw the figure
    area_plot = px.area(
        data_frame=area_network,
        x="Timestep",
        y=variable,
        color="Mobility mode",
        color_discrete_sequence=px.colors.sequential.Plasma_r,
        hover_data=["Mobility flow"],
        title="Mobility mode time series",
    )
    # Customize the hovers, title and yaxis
    area_plot.update_layout(
        hoverlabel=hovers,
        title_x=0.11,
        yaxis_title=f"{variable} ({aq_units[variable]})",
    )

    third_plot = area_plot
    third_plot_style = {"padding-top": "3vh"}

    # Return the figures, styles and location
    return first_plot, first_plot_style, second_plot, second_plot_style, third_plot, third_plot_style, location

# Reset the AQ heatmap
@app.callback(
    Output("aq-one", "clickData"),
    [Input("reset-button-aq", "n_clicks")])
def resetAirQualityLocation(reset):
    return None

####-----------------------------------------

## Livability

####-----------------------------------------

app.run(debug=True)